In [1]:
import scanpy as sc
import scvelo as scv
import scarches as sca
import numpy as np
import pandas as pd
from velovi import preprocess_data
import scarches as sca

 captum (see https://github.com/pytorch/captum).


KeyboardInterrupt: 

In [ ]:
#Change gene annotations pbmc

annot = sc.queries.biomart_annotations(
        "hsapiens",
        ["ensembl_gene_id", "start_position", "end_position", "chromosome_name"],
    ).set_index("ensembl_gene_id")

In [ ]:
adata.var[annot.columns] = annot

In [2]:
dataset="hindbrain"

In [3]:
if dataset=="pancreas":
    input_adata_file = "data/pancreas/endocrinogenesis_day15.h5ad"
    output_adata_file = "data/pancreas/pancreas_data_annotations.h5ad"
elif dataset=="dentategyrus":
    input_adata_file = "data/dentategyrus/10X43_1.h5ad"
    output_adata_file = "data/dentategyrus/dentategyrus_data_annotations.h5ad"
elif dataset=="hindbrain":
    input_adata_file = "data/hindbrain/Hindbrain_GABA_Glio.h5ad"
    output_adata_file = "data/hindbrain/hindbrain_data_annotations.h5ad"
elif dataset=="pbmc":
    input_adata_file = "data/pbmc/pbmc_10k.h5ad"
    output_adata_file = "data/pbmc/pbmc_data_annotations.h5ad"
    

In [4]:
import scanpy as sc
adata=sc.read(input_adata_file)

OSError: Unable to open file (truncated file: eof = 4194304, sblock->base_addr = 0, stored_eof = 259329434)

In [3]:
adata.layers["spliced"]=adata.layers["spliced"].todense()
adata.layers["unspliced"]=adata.layers["unspliced"].todense()

KeyError: 'spliced'

In [6]:
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

adata = preprocess_data(adata)

Filtered out 20801 genes that are detected 20 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 2000 highly variable genes.
Logarithmized X.
computing neighbors


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


    finished (0:00:04) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


In [7]:
data_path='data/annotations/'
sca.utils.add_annotations(adata, [data_path+'PanglaoDB_markers_27_Mar_2020.gmt',data_path+'reactome.gmt'], min_genes=5, clean=True)
#Remove all genes not present in annotations
adata._inplace_subset_var(adata.varm['I'].sum(1)>0)

sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    batch_key="batch",
    subset=True)

#Filter out terms with less than 12 genes 
select_terms = adata.varm['I'].sum(0)>12
adata.uns['terms'] = np.array(adata.uns['terms'])[select_terms].tolist()
adata.varm['I'] = adata.varm['I'][:, select_terms]

In [18]:
adata.write_h5ad(output_adata_file)